In [4]:
import cv2
import numpy as np
from importnb import Notebook
import matplotlib.pyplot as plt
import numpy.fft as fft
import numpy.matlib

In [5]:
with Notebook(): 
        import utils

In [6]:
img = utils.loadImage('resources/blu.png', 'gray')
print(img)

image resources/blu.png loaded
[[35 35 34 ... 36 37 37]
 [34 34 34 ... 37 38 38]
 [34 34 33 ... 40 40 40]
 ...
 [49 49 49 ... 53 57 61]
 [49 49 49 ... 54 58 62]
 [49 49 49 ... 54 58 63]]
